In [1]:
from hvo_sequence import midi_to_hvo_sequence
from hvo_sequence import ROLAND_REDUCED_MAPPING
GROOVE_MAPPING = {f'groove': [x for x in range(128)]}

import pandas as pd
from tqdm import tqdm
import os
import numpy as np
import pickle
import bz2
from sklearn.model_selection import train_test_split

In [2]:
# Collecting desired tracks
#   - Has genre
#   - Contains instrument and Drums
#   - Constant 4/4 Time Signature

dataset_genre_drums_path = 'F:/Nil/Instrument2Groove2Drum/data/lmd_separated/has_genre/has_drums/'

print(len(os.listdir(dataset_genre_drums_path)))

11462


In [3]:
instrument_name = 'piano'
instrument_class = '_[Piano'
preprocessed_data_path = 'data/i2dgd/piano2drum.bz2pickle'

In [4]:
genre_annotation_json_path = 'F:/Nil/Instrument2Groove2Drum/data/lmd_matched_genre.json'

genre_annotation_no_duplicates = pd.read_json(genre_annotation_json_path).drop_duplicates(subset=['trackId'], keep='first').set_index('filename')

In [5]:
file_paths = {}

for root, dirs, files in os.walk(dataset_genre_drums_path):
    lakh_id = os.path.basename(root)
    if lakh_id in genre_annotation_no_duplicates.index:
        md = pd.read_csv(os.path.join(root, f"{lakh_id}.csv"))
        if (md.iloc[:, -2:] == 4).all().all():
            genre = genre_annotation_no_duplicates.loc[lakh_id]['genre']
            drums = None # Single file containing _[Drums]_
            instruments = [] 
            for file in files:
                if "_[Drums]_" in file:
                    drums = os.path.join(root, file)
                elif instrument_class in file:
                    instruments.append(os.path.join(root, file))
            if drums is not None and len(instruments):
                file_paths[lakh_id.replace(".mid", "")] = (drums, instruments, genre)

In [6]:
len(file_paths)

2279

In [7]:
file_paths

{'000c004a21a44e2c80f3f549f4abc8b5': ('F:/Nil/Instrument2Groove2Drum/data/lmd_separated/has_genre/has_drums/000c004a21a44e2c80f3f549f4abc8b5.mid\\[]_[Drums]_[tqT]_[vqF].mid',
  ['F:/Nil/Instrument2Groove2Drum/data/lmd_separated/has_genre/has_drums/000c004a21a44e2c80f3f549f4abc8b5.mid\\[]_[Electric Piano 2]_[Piano]_[tqF]_[vqF].mid'],
  'Pop'),
 '000c923c18c62332503050809ea3eb8f': ('F:/Nil/Instrument2Groove2Drum/data/lmd_separated/has_genre/has_drums/000c923c18c62332503050809ea3eb8f.mid\\[Drums]_[Drums]_[tqF]_[vqF].mid',
  ['F:/Nil/Instrument2Groove2Drum/data/lmd_separated/has_genre/has_drums/000c923c18c62332503050809ea3eb8f.mid\\[Electric Piano]_[Electric Piano 1]_[Piano]_[tqF]_[vqF].mid'],
  'Pop'),
 '001784858d9e607c0db054e68cab8c6c': ('F:/Nil/Instrument2Groove2Drum/data/lmd_separated/has_genre/has_drums/001784858d9e607c0db054e68cab8c6c.mid\\[ATLANTIS]_[Drums]_[tqT]_[vqF].mid',
  ['F:/Nil/Instrument2Groove2Drum/data/lmd_separated/has_genre/has_drums/001784858d9e607c0db054e68cab8c6c.mi

In [8]:
def merge_hvo_sequences(hvo_sequences):
    assert all(hvo_seq.hvo.shape == hvo_sequences[0].hvo.shape for hvo_seq in hvo_sequences)

    shape = hvo_sequences[0].hvo.shape
    merged_arr = np.zeros_like(hvo_sequences[0].hvo)

    for i in range(shape[0]):
        max_val = -np.inf
        max_row = None
        for hvo_seq in hvo_sequences:
            if hvo_seq.hvo[i, 1] > max_val:
                max_val = hvo_seq.hvo[i, 1]
                max_row = hvo_seq.hvo[i, :]
        merged_arr[i, :] = max_row

    return merged_arr

In [9]:
training_data = {}

inputs = []
outputs = []
filenames = []
genres = []
inputs_hvo_seqs = []
outputs_hvo_seqs = []

for lakh_id, values in tqdm(file_paths.items()):
    drum_path, instrument_paths, genre = values

    _2bar_hvo_seqs_drums = []
    _2bar_hvo_seqs_drum_grooves = []
    _2bar_hvo_seqs_instrument_grooves = []

    common_md = {
            'style_primary': genre,
            'lakh_id': lakh_id
    }

    # Drum HVO
    drum_hvo = midi_to_hvo_sequence(filename=drum_path, drum_mapping=ROLAND_REDUCED_MAPPING, beat_division_factors=[4])
    drum_hvo.metadata['instrument'] = 'drums'
    drum_hvo.metadata['midi_path'] = drum_path
    drum_hvo.metadata.update(common_md)
    # Adjusting lenght to be multiple of 16
    drum_n_bars = drum_hvo.number_of_steps / 16
    drum_hvo.adjust_length(int(np.round(drum_n_bars) * 16))

    # Drum Groove HVO
    drum_groove_hvo = midi_to_hvo_sequence(filename=drum_path, drum_mapping=GROOVE_MAPPING, beat_division_factors=[4])
    drum_groove_hvo.metadata['instrument'] = 'drums'
    drum_groove_hvo.metadata['midi_path'] = drum_path
    drum_groove_hvo.metadata.update(common_md)
    # Adjusting lenght to be multiple of 16
    drum_groove_hvo.adjust_length(int(np.round(drum_n_bars) * 16))

    # instrument Grooves HVOs 
    all_instrument_grooves_hvo = []
    for instrument_path in instrument_paths:
        instrument_groove_hvo = midi_to_hvo_sequence(filename=instrument_path, drum_mapping=GROOVE_MAPPING, beat_division_factors=[4])
        instrument_groove_hvo.metadata['instrument'] = instrument_name
        instrument_groove_hvo.metadata.update(common_md)
        # Adjusting lenght to be multiple of 16
        instrument_groove_hvo.adjust_length(int(np.round(drum_n_bars) * 16))
        all_instrument_grooves_hvo.append(instrument_groove_hvo)


    # Splitting into 2 bar segments
    for w_start in range(int(drum_n_bars)-1):
        start_step = int(w_start * 16)
        end_step = start_step + 32
        
        # Drum HVOs
        seg_drum_hvo_seq = drum_hvo.copy_empty()
        seg_drum_hvo_seq.hvo = drum_hvo.hvo[start_step:end_step]

        # Drum Grooves HVOs
        seg_drum_groove_hvo_seq = drum_groove_hvo.copy_empty()
        seg_drum_groove_hvo_seq.hvo = drum_groove_hvo.hvo[start_step:end_step]

        # instrument Grooves HVOs
        all_seq_instrument_hvo_seqs = []
        for instrument_groove_hvo in all_instrument_grooves_hvo:
            seg_instrument_groove_hvo_seq = instrument_groove_hvo.copy_empty()
            seg_instrument_groove_hvo_seq.hvo = instrument_groove_hvo.hvo[start_step:end_step]
            all_seq_instrument_hvo_seqs.append(seg_instrument_groove_hvo_seq)
        # Merging instrument sequences into a single HVO
        seg_instrument_groove_hvo_seq = all_seq_instrument_hvo_seqs[0].copy_empty()
        seg_instrument_groove_hvo_seq.hvo = merge_hvo_sequences(all_seq_instrument_hvo_seqs)

        if np.any(seg_instrument_groove_hvo_seq.hits) and np.any(seg_drum_groove_hvo_seq.hits):
            _2bar_hvo_seqs_drums.append(seg_drum_hvo_seq)
            _2bar_hvo_seqs_drum_grooves.append(seg_drum_groove_hvo_seq)
            _2bar_hvo_seqs_instrument_grooves.append(seg_instrument_groove_hvo_seq)

    # Appending data into corresponding lists
    for i in range(len(_2bar_hvo_seqs_drums)):
        inputs.append(_2bar_hvo_seqs_instrument_grooves[i].hvo)
        inputs_hvo_seqs.append(_2bar_hvo_seqs_instrument_grooves[i])
        outputs.append(_2bar_hvo_seqs_drum_grooves[i].hvo)
        outputs_hvo_seqs.append(_2bar_hvo_seqs_drum_grooves[i])
        filenames.append(lakh_id)
        genres.append(genre)


100%|██████████| 2279/2279 [2:12:04<00:00,  3.48s/it]   


In [10]:
len(inputs)

170283

In [11]:
# Separation of the data into 80% train - 20% test.

assert len(inputs) == len(outputs) == len(genres) == len(inputs_hvo_seqs) == len(outputs_hvo_seqs)

indexes = list(range(len(inputs)))

train_indexes, test_indexes = train_test_split(indexes, test_size=0.2, random_state=42)

In [12]:
print(f"Total number of samples (2bar sequences): {len(indexes)}")
print(f"Number of train samples: {len(train_indexes)}")
print(f"Number of test samples: {len(test_indexes)}")

Total number of samples (2bar sequences): 170283
Number of train samples: 136226
Number of test samples: 34057


In [13]:
data = {
    "train": {
        "inputs": [inputs[i] for i in train_indexes],
        "outputs": [outputs[i] for i in train_indexes],
        "outputs_hvo_seqs": [outputs_hvo_seqs[i] for i in train_indexes],
        "filenames": [filenames[i] for i in train_indexes],
        "style_primary": [genres[i] for i in train_indexes]
    },
    "test": {
        "inputs": [inputs[i] for i in test_indexes],
        "outputs": [outputs[i] for i in test_indexes],
        "outputs_hvo_seqs": [outputs_hvo_seqs[i] for i in test_indexes],
        "filenames": [filenames[i] for i in test_indexes],
        "style_primary": [genres[i] for i in test_indexes]
    },
}

In [14]:
with bz2.BZ2File(preprocessed_data_path, 'wb') as file:
    pickle.dump(data, file)